In [1]:
import fitz
from glob import *
import os
import pdfplumber
import time
from tqdm.notebook import tqdm

In [2]:
PATH_NOTES = r"G:\Esp-mission-Patrimoine\documentLibrary\Habitat_Amenagement\OBSERVATOIRE\02_ACTIVITES\ETUDES\ETUDES_ADIL\NOTES"
chemin_notes = sorted(glob(PATH_NOTES + r"\*.pdf", ), key=lambda x: int(x.split("\\")[-1].split("_")[0]))
# Pour le moment
chemin_notes.remove('G:\\Esp-mission-Patrimoine\\documentLibrary\\Habitat_Amenagement\\OBSERVATOIRE\\02_ACTIVITES\\ETUDES\\ETUDES_ADIL\\NOTES\\30_airbnb_JURIDIQUE_2024.pdf')
nom_notes = [chemin.split('\\')[-1].split('.')[0] for chemin in chemin_notes]
numero_notes = [int(chemin.split('\\')[-1].split("_")[0]) for chemin in chemin_notes]
chemin_notes[-2:], nom_notes[-2:], numero_notes[-2:]

(['G:\\Esp-mission-Patrimoine\\documentLibrary\\Habitat_Amenagement\\OBSERVATOIRE\\02_ACTIVITES\\ETUDES\\ETUDES_ADIL\\NOTES\\29_Loyers_2024.pdf',
  'G:\\Esp-mission-Patrimoine\\documentLibrary\\Habitat_Amenagement\\OBSERVATOIRE\\02_ACTIVITES\\ETUDES\\ETUDES_ADIL\\NOTES\\30_airbnb_DIAG_2024.pdf'],
 ['29_Loyers_2024', '30_airbnb_DIAG_2024'],
 [29, 30])

In [3]:
def remove_occurences_3(text):
    text_copy = text
    li = []
    # Trouve les suites de charactères à 3 occurences et rajoute les indices dans li (pas le premier)
    for i, _ in enumerate(text_copy[:-2]):
        if text_copy[i:i+3] == text_copy[i]*3:
            j=1
            while text_copy[i+j] == text_copy[i]:
                j += 1
                li.append(i+j-1)

    # Supprime les indices qui sont au milieu d'une suite
    lb = set(li)
    for el in set(li):
        if el + 1 in set(li) and el - 1 in set(li):
            lb.remove(el)

    # Supprime les suites
    lc = sorted(list(lb))
    for i in range(0, len(lc), 2):
        text_copy = text_copy.replace(text[lc[i]:lc[i+1]+1], '')
        # print(text[lc[i]:lc[i+1]+1], 'remplacée par ""')
    
    return text_copy

In [3]:
prepro_dir = 'preprocessing'
os.makedirs(os.path.join(PATH_NOTES, prepro_dir), exist_ok=True)
texte_dir = 'texte'

In [ ]:
# PATH_NOTES / preprocessing / "nom_note" / texte
for chemin, nom, numero in tqdm(zip(chemin_notes, nom_notes, numero_notes), desc="Processing Notes"):    
    pdf = pdfplumber.open(chemin)
    note_dir = f"{nom.split('.')[0]}"
    page_dir = os.path.join(PATH_NOTES, prepro_dir, note_dir, texte_dir)
    os.makedirs(page_dir, exist_ok=True)

    for idx, page in tqdm(enumerate(pdf.pages), desc="Processing Pages", leave=False):
        text = page.extract_text()
        text_clean = remove_occurences_3(text)
        with open(f"{page_dir}/page_{idx+1}.txt", "w", encoding='utf-8') as f:
            f.write(text_clean)


In [43]:
# VERIF nombre de pages
with open(os.path.join(PATH_NOTES, prepro_dir, 'meta/nb_page.txt'), "r") as f:
    texte = f.readlines()
    texte_c = [(line.split(' ')[0],int(line.split(' ')[-1])) for line in texte]
    dict_nb_page = {line.split(' ')[0]:int(line.split(' ')[-1]) for line in texte}

for note in glob(os.path.join(PATH_NOTES, prepro_dir, '*')):
    if 'meta' in note:
        continue
    numero_notes = note.split('\\')[-1].split("_")[0]
    print("num note :", numero_notes, "nb page.txt :", len(glob(os.path.join(note, texte_dir, '*.txt'))), "vrai nb page :", dict_nb_page[numero_notes])
    if dict_nb_page[numero_notes] != len(glob(os.path.join(note, texte_dir, '*.txt'))):
        print('*'*10)
        print("!!! Erreur dans le nombre de pages pour la note :", numero_notes)
        print("nb page.txt :", len(glob(os.path.join(note, texte_dir, '*.txt'))), "vrai nb page :", dict_nb_page[numero_notes])
        print('*'*10)

num note : 08 nb page.txt : 27 vrai nb page : 27
num note : 04 nb page.txt : 12 vrai nb page : 12
num note : 30 nb page.txt : 21 vrai nb page : 21
num note : 05 nb page.txt : 16 vrai nb page : 16
num note : 29 nb page.txt : 23 vrai nb page : 23
num note : 24 nb page.txt : 11 vrai nb page : 11
num note : 19 nb page.txt : 20 vrai nb page : 20
num note : 15 nb page.txt : 33 vrai nb page : 33
num note : 14 nb page.txt : 27 vrai nb page : 27
num note : 01 nb page.txt : 23 vrai nb page : 23
num note : 27 nb page.txt : 24 vrai nb page : 24
num note : 11 nb page.txt : 30 vrai nb page : 30
num note : 25 nb page.txt : 23 vrai nb page : 23
num note : 07 nb page.txt : 15 vrai nb page : 15
num note : 21 nb page.txt : 18 vrai nb page : 18
num note : 20 nb page.txt : 17 vrai nb page : 17
num note : 13 nb page.txt : 27 vrai nb page : 27
num note : 18 nb page.txt : 22 vrai nb page : 22
num note : 02 nb page.txt : 6 vrai nb page : 6
num note : 12 nb page.txt : 29 vrai nb page : 29
num note : 16 nb page.

In [ ]:
# VERIF pages non vides
for note_dir_path in glob(os.path.join(PATH_NOTES, prepro_dir, '*')):
    note_basename = os.path.basename(note_dir_path)

    if 'meta' in note_basename:
        continue

    numero_notes = note_basename.split("_")[0]
    print(f"NOTE {numero_notes}") # Using f-string for cleaner formatting

    for page_file_path in glob(os.path.join(note_dir_path, texte_dir, '*.txt')):
        page_filename = os.path.basename(page_file_path)
        page_deleted_this_iteration = False # Flag to track if the current page gets deleted

        try:
            with open(page_file_path, "r", encoding='utf-8') as f:
                texte = f.read()
        except IOError as e:
            print(f"Error reading file {page_filename}: {e}")
            continue # Skip to the next page

        if len(texte) < 10:
            print('*'*10)
            print(f"!!! Erreur dans le nombre de pages pour la note : {numero_notes}")
            print(f"page : {page_filename}")
            print(f"texte : {texte}")
            print('*'*10)

            # Ask for user confirmation and process immediately
            # .strip() handles accidental spaces, .lower() makes comparison case-insensitive
            choix_suppression = input("Voulez-vous supprimer la page ? (o/n) : ").strip().lower()
            if choix_suppression == 'o':
                try:
                    os.remove(page_file_path)
                    print(f"Page '{page_filename}' supprimée.")
                    page_deleted_this_iteration = True
                except OSError as e:
                    print(f"Error deleting file {page_filename}: {e}")
                    # If deletion fails, it's considered not deleted for the next message
            # No explicit else here for "page non supprimée" yet,
            # it will be handled by the final check.

        # This final check maintains the original logic:
        # "page non supprimée" is printed if:
        # 1. The page was long (len(texte) >= 10)
        # 2. The page was short, but the user chose not to delete it.
        # 3. The page was short, user chose to delete, but deletion failed.
        if choix_suppression != 'o' and not page_deleted_this_iteration:
            print(f"Page '{page_filename}' non supprimée.")